__Общая задача:__ создать скрипт для формирования витрины на основе логов web-сайта.

In [3]:
import pandas as pd        # импортируем необходимые библиотеки
import numpy as np
import psycopg2
from datetime import datetime
import re

In [4]:
# Import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql.functions import *
import pyspark.sql.functions as F


In [5]:
# Create SparkSession 
spark = SparkSession.builder \
  .master("local[1]") \
  .appName("SparkFirst") \
  .config("spark.executor.memory", "10g")\
  .config("spark.executor.cores", 5) \
  .config("spark.dynamicAllocation.enabled", "true") \
  .config("spark.dynamicAllocation.maxExecutors", 5) \
  .config("spark.shuffle.service.enabled", "true") \
  .getOrCreate()

In [6]:
base_df = spark.read.text('access.log')
base_df.printSchema()

root
 |-- value: string (nullable = true)



In [5]:
base_df.show(3, truncate=False)


+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                                                                                                                     |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|54.36.149.41 - - [22/Jan/2019:03:56:14 +0330] "G

In [6]:
(base_df.filter(base_df['value'].isNull()).count()) # проверим есть ли нулевые строки в исходном датафрейме

0

In [9]:
sample_logs = [item['value'] for item in base_df.take(5)]  # выберем небольшую выборку из основного датафрейма для проверки регулярок
sample_logs

['54.36.149.41 - - [22/Jan/2019:03:56:14 +0330] "GET /filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,27|%DA%A9%D9%85%D8%AA%D8%B1%20%D8%A7%D8%B2%205%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,p53 HTTP/1.1" 200 30577 "-" "Mozilla/5.0 (compatible; AhrefsBot/6.1; +http://ahrefs.com/robot/)" "-"',
 '31.56.96.51 - - [22/Jan/2019:03:56:16 +0330] "GET /image/60844/productModel/200x200 HTTP/1.1" 200 5667 "https://www.zanbil.ir/m/filter/b113" "Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36" "-"',
 '31.56.96.51 - - [22/Jan/2019:03:56:16 +0330] "GET /image/61474/productModel/200x200 HTTP/1.1" 200 5379 "https://www.zanbil.ir/m/filter/b113" "Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36" "-"',
 '40.77.167.129 - - [22/Jan/2019:03:56:17 +0330] "GET /image/14925/productModel/100x100 HTTP/

При помощи регулярных выражений спарсим необходимые данные. Из лога я взяла не все данные, только хост, время, метод, протокол, статус и самое главное строку с User agent-информацией.

In [10]:
host_pattern = r'(^\S+\.[\S+\.]+\S+)\s'              #регулярное выражение для хоста
hosts = [re.search(host_pattern, item).group(1)
           if re.search(host_pattern, item)
           else 'no match'
           for item in sample_logs]
hosts

['54.36.149.41', '31.56.96.51', '31.56.96.51', '40.77.167.129', '91.99.72.15']

In [11]:
ts_pattern = r'\[([^:]+):(\d+:\d+:\d+) ([^\]]+)\]'   #регулярное выражение для даты
timestamps = [re.search(ts_pattern, item).group(1) for item in sample_logs]
timestamps

['22/Jan/2019', '22/Jan/2019', '22/Jan/2019', '22/Jan/2019', '22/Jan/2019']

In [12]:
method_uri_protocol_pattern = r'\"(\S+) (.*?) (\S+)\"'     #регулярное выражение для метода
method_uri_protocol = [re.search(method_uri_protocol_pattern, item).groups()
               if re.search(method_uri_protocol_pattern, item)
               else 'no match'
              for item in sample_logs]
method_uri_protocol

[('GET',
  '/filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,27|%DA%A9%D9%85%D8%AA%D8%B1%20%D8%A7%D8%B2%205%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,p53',
  'HTTP/1.1'),
 ('GET', '/image/60844/productModel/200x200', 'HTTP/1.1'),
 ('GET', '/image/61474/productModel/200x200', 'HTTP/1.1'),
 ('GET', '/image/14925/productModel/100x100', 'HTTP/1.1'),
 ('GET',
  '/product/31893/62100/%D8%B3%D8%B4%D9%88%D8%A7%D8%B1-%D8%AE%D8%A7%D9%86%DA%AF%DB%8C-%D9%BE%D8%B1%D9%86%D8%B3%D9%84%DB%8C-%D9%85%D8%AF%D9%84-PR257AT',
  'HTTP/1.1')]

In [13]:
status_pattern = r'\s(\d{3})\s'                     #регулярное выражение для ответа
status = [re.search(status_pattern, item).group(1) for item in sample_logs]
print(status)

['200', '200', '200', '200', '200']


In [14]:
user_agent_top=r'\S+\S+\".\"([^\(]+)\((.*)\)'      #регулярное выражение для начала UA-строки
user_agent = [re.search(user_agent_top, item).group(1) for item in sample_logs]
user_agent

['Mozilla/5.0 ',
 'Mozilla/5.0 ',
 'Mozilla/5.0 ',
 'Mozilla/5.0 ',
 'Mozilla/5.0 ']

In [15]:
user_agent_end=r'\((.*) '                       #регулярнео выражение для окончания UA-строки
user_agent = [re.search(user_agent_end, item).group(1) for item in sample_logs]
user_agent

['compatible; AhrefsBot/6.1; +http://ahrefs.com/robot/)"',
 'Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36"',
 'Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36"',
 'compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)"',
 'Windows NT 6.2; Win64; x64; rv:16.0)Gecko/16.0 Firefox/16.0"']

In [16]:
from pyspark.sql.functions import regexp_extract       # теперь объединяем все вместе

logs_df = base_df.select(regexp_extract('value', host_pattern, 1).alias('host'),
                         regexp_extract('value', ts_pattern, 1).alias('timestamp'),
                         regexp_extract('value', method_uri_protocol_pattern, 1).alias('method'),
                         regexp_extract('value', method_uri_protocol_pattern, 2).alias('endpoint'),
                         regexp_extract('value', method_uri_protocol_pattern, 3).alias('protocol'),
                         regexp_extract('value', status_pattern, 1).cast('integer').alias('status'),
                         regexp_extract('value', user_agent_top, 1).alias('user_agent_top'),
                         regexp_extract('value', user_agent_end, 1).alias('user_agent_end'))
logs_df.show(10, truncate=True)
print((logs_df.count(), len(logs_df.columns)))

+-------------+-----------+------+--------------------+--------+------+--------------+--------------------+
|         host|  timestamp|method|            endpoint|protocol|status|user_agent_top|      user_agent_end|
+-------------+-----------+------+--------------------+--------+------+--------------+--------------------+
| 54.36.149.41|22/Jan/2019|   GET|/filter/27|13%20%...|HTTP/1.1|   200|  Mozilla/5.0 |compatible; Ahref...|
|  31.56.96.51|22/Jan/2019|   GET|/image/60844/prod...|HTTP/1.1|   200|  Mozilla/5.0 |Linux; Android 6....|
|  31.56.96.51|22/Jan/2019|   GET|/image/61474/prod...|HTTP/1.1|   200|  Mozilla/5.0 |Linux; Android 6....|
|40.77.167.129|22/Jan/2019|   GET|/image/14925/prod...|HTTP/1.1|   200|  Mozilla/5.0 |compatible; bingb...|
|  91.99.72.15|22/Jan/2019|   GET|/product/31893/62...|HTTP/1.1|   200|  Mozilla/5.0 |Windows NT 6.2; W...|
|40.77.167.129|22/Jan/2019|   GET|/image/23488/prod...|HTTP/1.1|   200|  Mozilla/5.0 |compatible; bingb...|
|40.77.167.129|22/Jan/2019| 

Проверим схему нашего датафрейма.

In [17]:
logs_df.printSchema()

root
 |-- host: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- method: string (nullable = true)
 |-- endpoint: string (nullable = true)
 |-- protocol: string (nullable = true)
 |-- status: integer (nullable = true)
 |-- user_agent_top: string (nullable = true)
 |-- user_agent_end: string (nullable = true)



Кэшируем logs_df для дальнейшего анализа:

In [18]:
logs_df.cache()

DataFrame[host: string, timestamp: string, method: string, endpoint: string, protocol: string, status: int, user_agent_top: string, user_agent_end: string]

Приступим к проведению анализа и построению витрины:
- Суррогатный ключ устройства

- Название устройства

- Количество пользователей

- Доля пользователей данного устройства от общего числа пользователей

- Количество совершенных действий для данного устройства

- Доля совершенных действий с данного устройства относительно других устройств

- Список из 5 самых популярных браузеров, используемых на данном устройстве различными пользователями, с указанием доли использования для данного браузера относительно остальных браузеров 

- Количество ответов сервера, отличных от 200 на данном устройстве

- Для каждого из ответов сервера, отличных от 200, сформировать поле, в котором будет содержаться количество ответов данного типа

Нам необходимо дополнительно проставить флаг или признак устройства.

In [22]:
@F.udf
def determine (x):            # объявим пользовательскую функцию для определения устройства
    if 'Android' in str(x): 
        return 'Android'
    if 'iPhone' in str(x):
        return 'iPhone'
    if 'iPad' in str(x):
        return 'iPad'
    if 'iPod' in str(x):
        return 'iPod'
    else:
        return 'Desctop'                # и протестируем строку 
    

In [23]:
determine("LinuxMozilla/5.0 (Linux; iPod; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36")

'iPod'

In [ ]:
sample = logs_df.toPandas().loc[0:5, ['host', 'method','status','user_agent_end']]
sample

In [ ]:
logs_df=logs_df.withColumn("Device", lit(determine(F.col("user_agent_end"))))\
         .show(3)                                  # применим функцию и создадим новый столбец с устройством

In [1]:
sample = logs_df.iloc[[0, 25, 100, 159, 300]]

NameError: name 'logs_df' is not defined

Количество пользователей - это количество уникальных хостов:

In [17]:
total_cust = logs_df.select(count_distinct("host")).show()

+--------------------+
|count(DISTINCT host)|
+--------------------+
|              258606|
+--------------------+



Количество пользователей устройства - это пользователи, сгруппированные по устройству:

In [ ]:
device_users=logs_df.select('host').groupby('Device').count().sort('count', ascending=False).show()

Количество совершенных действий для данного устройства - это группировка по колонке и подсчет методов (GET, POST):

In [ ]:
device_actions=logs_df.select('method').groupby('Device').count().show()

Всего действий - это подсчет значений колонки методов:

In [ ]:
total_actions = logs_df.select('method').count().show()

Доля совершенных действий с данного устройства относительно других устройств - это 

Список из 5 самых популярных браузеров, используемых на данном устройстве различными пользователями, с указанием доли использования для данного браузера относительно остальных браузеров - это 

In [ ]:
total_status=logs_df.groupBy('status').count().sort('status', ascending=False).show() 

Количество ответов сервера, отличных от 200 на данном устройстве:

Для каждого из ответов сервера, отличных от 200, сформировать поле, в котором будет содержаться количество ответов данного типа:

In [ ]:
db_con = psycopg2.connect(database='testdb',     # создадим подключение к созданной базе данных exrate
                        user='postgres',
                        password='pass',
                        host='localhost',
                        port=5435)
cur = db_con.cursor()

In [ ]:
cur.execute(""" CREATE TABLE btc_Jan2023(
    id SERIAL PRIMARY KEY,
    date DATE,
    val_id VARCHAR,
    base_rate DECIMAL,
    val_base VARCHAR
)
""") 
db_con.commit()        

In [ ]:
cur.close()
db_con.close()